In [1]:
import pandas as pd

In [7]:
data = pd.read_csv('telecom_churn.csv')
data

,churn,accountweeks,contractrenewal,dataplan,datausage,custservcalls,daymins,daycalls,monthlycharge,overagefee,roammins
0,0,128,1,1,2.70,1,265.1,110,89.0,9.87,10.0
1,0,107,1,1,3.70,1,161.6,123,82.0,9.78,13.7
2,0,137,1,0,0.00,0,243.4,114,52.0,6.06,12.2
3,0,84,0,0,0.00,2,299.4,71,57.0,3.10,6.6
4,0,75,0,0,0.00,3,166.7,113,41.0,7.42,10.1
...,...,...,...,...,...,...,...,...,...,...,...
3328,0,192,1,1,2.67,2,156.2,77,71.7,10.78,9.9
3329,0,68,1,0,0.34,3,231.1,57,56.4,7.67,9.6
3330,0,28,1,0,0.00,2,180.8,109,56.0,14.44,14.1
3331,0,184,0,0,0.00,2,213.8,105,50.0,7.98,5.0


In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3333 entries, 0 to 3332
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   churn            3333 non-null   int64  
 1   accountweeks     3333 non-null   int64  
 2   contractrenewal  3333 non-null   int64  
 3   dataplan         3333 non-null   int64  
 4   datausage        3333 non-null   float64
 5   custservcalls    3333 non-null   int64  
 6   daymins          3333 non-null   float64
 7   daycalls         3333 non-null   int64  
 8   monthlycharge    3333 non-null   float64
 9   overagefee       3333 non-null   float64
 10  roammins         3333 non-null   float64
dtypes: float64(5), int64(6)
memory usage: 286.6 KB


In [10]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

for column in data.select_dtypes(include=['object']).columns:
    data[column] = le.fit_transform(data[column])

correlation_matrix = data.corr()

target_correlations = correlation_matrix['churn']
positive_correlations = target_correlations[target_correlations > 0].index
positive_features = positive_correlations.drop('churn')
filtered_df = data[positive_features]
filtered_df = filtered_df.join(data['churn'])

print(filtered_df.columns)

from sklearn.model_selection import train_test_split

X = filtered_df.drop('churn', axis=1)  
y = data['churn'] 

import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=100)

model = xgb.XGBClassifier(eval_metric='logloss')

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

# Display results
print(f"Accuracy: {accuracy:.2f}")
print("Confusion Matrix:")
print(conf_matrix)
print("Classification Report:")
print(class_report)

Index(['accountweeks', 'custservcalls', 'daymins', 'daycalls', 'monthlycharge',
       'overagefee', 'roammins', 'churn'],
      dtype='object')
Accuracy: 0.92
Confusion Matrix:
[[853  25]
 [ 51  71]]
Classification Report:
              precision    recall  f1-score   support

           0       0.94      0.97      0.96       878
           1       0.74      0.58      0.65       122

    accuracy                           0.92      1000
   macro avg       0.84      0.78      0.80      1000
weighted avg       0.92      0.92      0.92      1000



In [11]:
model.save_model('model3.json')